In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [2]:
IMG_SIZE = 150

In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)

        x = torch.randn(IMG_SIZE,IMG_SIZE).view(-1,1,IMG_SIZE,IMG_SIZE)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512) #flattening step 
        self.fc2 = nn.Linear(512, 28)

    def convs(self, x):
        drp = nn.Dropout(0.25)
        
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = drp(x)
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = drp(x)
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        x = drp(x)

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=36992, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=28, bias=True)
)


In [5]:
def loading_model():

    model_save_path = "C:/Meh/jupyter nb/ML Minor Project/model save/CNNmodel_new.pth"
    #model_save_path = "D:/CNNmodel_new.pth"
    mynet = torch.load(model_save_path)
    return mynet

MyNN = loading_model()

In [6]:
LABELS = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G', 7:'H', 8:'I', 9:'J', 10:'K', 11:'L', 12:'M', 13:'N', 14:'O', 15:'P', 16:'Q', 17:'R', 18:'S', 19:'T', 20:'U', 21:'V', 22:'W', 23:'X', 24:'Y', 25:'Z', 26:'_', 27:'*'}

In [19]:
vid = cv2.VideoCapture(0)
frame_list = []
text = 'text: '
guess = ''
while True:
    _, frame = vid.read()
    frame = cv2.flip(frame, 1)
    
    cv2.rectangle(frame, (320-1, 109), (620+1, 419), (255,0,0) ,2)
    # seperating cropped image from the frame
    
    crop = frame[110:410, 320:620]

    #cv2.imshow("Frame", frame)
    
    #converting bgr colored image to grayscale
    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    
    #blur = cv2.GaussianBlur(gray,(7,7),2)
    blur = cv2.GaussianBlur(gray,(15,15),2)
    
    minValue = 240
    #minValue = 130
    adap_thres = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
    ret, test_image = cv2.threshold(adap_thres, minValue, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    
    
    cv2.imshow("test", test_image)
    final_img = cv2.resize(test_image, (IMG_SIZE, IMG_SIZE))
    interrupt = -1  
    interrupt = cv2.waitKey(200)
    if interrupt & 0xFF == 27: # esc key
        break
        
    if interrupt:
        net_out = MyNN(torch.Tensor(final_img).view(-1, 1, 150, 150))
        arr = net_out[0].detach().numpy()
        for i in range(len(arr)):
            if int(arr[i])==1:
                guess = LABELS[i]
        cv2.putText(frame, str(guess), (450, 65), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 2)
        print(str(guess))
        
    if interrupt & 0xFF == ord('p'):
        text = text + str(guess)
    
    cv2.putText(frame, str(text), (10, 400), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,), 2)
    print(str(text))
    
    cv2.imshow("Frame", frame)
        
vid.release()
cv2.destroyAllWindows()

A
text: 
*
text: 
M
text: 
Y
text: 
C
text: 
M
text: 
A
text: 
M
text: 
P
text: 
A
text: 
A
text: 
_
text: 
_
text: 
_
text: 
P
text: 
A
text: 
A
text: 
A
text: 
Y
text: 
A
text: 
T
text: 
P
text: 
A
text: 
A
text: 
L
text: 
Y
text: 
A
text: 
N
text: 
H
text: 
A
text: 
A
text: 
A
text: 
A
text: 
C
text: 
A
text: 
C
text: 
A
text: 
C
text: 
Y
text: 
A
text: 
A
text: 
A
text: 
S
text: 
A
text: A
S
text: A
Y
text: A
S
text: A
E
text: A
S
text: A
Y
text: A
A
text: A
A
text: A
A
text: A
V
text: A
F
text: A
V
text: A
V
text: A
U
text: A
V
text: A
V
text: A
V
text: A
B
text: A
V
text: A
V
text: A
V
text: A
V
text: A
V
text: A
V
text: AV
V
text: AV
V
text: AV
V
text: AV
V
text: AV
V
text: AV
V
text: AV
V
text: AV
A
text: AV
B
text: AV
Z
text: AV
U
text: AV
_
text: AV
_
text: AV
J
text: AV
_
text: AV
Z
text: AV
Z
text: AV
Z
text: AV
Z
text: AV
Z
text: AV
F
text: AV
Z
text: AV
P
text: AV
Q
text: AV
*
text: AV
C
text: AV
C
text: AV
Z
text: AV
P
text: AV
P
text: AV
C
text: AV
*
text: AV
F
text: AV